In [1]:
import numpy as np

np.random.seed(42)
# get some synthetic data
n_data = 1000
n_features = 10
event_times = np.random.weibull(a=1, size=n_data).round(1)
censoring_times = np.random.lognormal(mean=1, sigma=1, size=n_data).round(1)
event_indicators = event_times < censoring_times
observed_times = np.minimum(event_times, censoring_times)

# get some synthetic predictions
n_times = 100
time_grid = np.linspace(1, 5, n_times)
predictions = np.random.rand(n_data, n_times)
# normalize the predictions to sum to 1, meaning the probability mass function
pmf = predictions / predictions.sum(axis=1)[:, None]
survival_curves = 1 - np.cumsum(pmf, axis=1)
predictions_cdf = np.cumsum(pmf, axis=1)

from SurvivalEVAL import SurvivalEvaluator

# initialize the evaluator
evaluator = SurvivalEvaluator(survival_curves, time_grid, observed_times, event_indicators, predict_time_method="Mean")
print("Successfully initialized the evaluator.")

# calculate the concordance index
cindex, _, _ = evaluator.concordance()
print(f"The concordance index is {cindex}.")

# calculate the MAE
mae = evaluator.mae(method="Hinge", weighted=False)
print(f"The MAE is {mae}.")

# calculate the d-calibration
pvalue, _ = evaluator.d_calibration(num_bins=10)
print(f"The p-value of the D-Calibration test is {pvalue}.")

# calculate the ibs
ibs = evaluator.integrated_brier_score(num_points=None, IPCW_weighted=False)
print(f"The IBS is {ibs}.")

# calculate the brier score
brier_score = evaluator.brier_score(target_time=2, IPCW_weighted=False)
print(f"The Brier score at time 2 is {brier_score}.")

# calculate the one calibration
one_cal = evaluator.one_calibration(target_time=2)
print(f"The one calibration at time 2 is {one_cal}.")

# calculate the AUC
auc = evaluator.auc(target_time=2)
print(f"The AUC at time 2 is {auc}.")

/home/weijiesun/anaconda3/envs/SurvivalEVAL/lib/python3.11/site-packages/SurvivalEVAL/Evaluator.py:73: UserWarning: The first time coordinate is not 0. A authentic survival curve should start from 0 with 100% survival probability. Adding 0 to the beginning of the time coordinates and 1 to the beginning of the predicted curves.
  warnings.warn(zero_pad_msg)


Successfully initialized the evaluator.
The concordance index is 0.49888651272540263.
The MAE is 1.9140860303268887.
The p-value of the D-Calibration test is 0.0.
The IBS is 0.23486649465374945.
The Brier score at time 2 is 0.4384085472343544.
The one calibration at time 2 is (np.float64(0.0), [np.float64(0.8416658045697618), np.float64(0.9125506477201393), np.float64(0.8604549571934075), np.float64(0.8712028221480657), np.float64(0.8028777459166332), np.float64(0.8006521087096243), np.float64(0.8792878488509428), np.float64(0.8843355258511713), np.float64(0.8749800947415178), np.float64(0.8904143234121934)], [np.float64(0.3033978647319048), np.float64(0.28558829989947854), np.float64(0.27652355887061814), np.float64(0.2692477225953314), np.float64(0.26277640749983067), np.float64(0.2566499589061098), np.float64(0.25043705610898803), np.float64(0.24345396390257626), np.float64(0.2329202779943213), np.float64(0.21381194240284354)]).
The AUC at time 2 is 0.4735244776574653.


# Test calibration_points_right_censor

In [2]:
from IntervalCensor import calibration_slope_right_censor
p_list = (0.1,0.3,0.5,0.7,0.9)
p_list, obsevation_list, slope = calibration_slope_right_censor(
    event_indicators=event_indicators,     # bool
    observed_times=observed_times,    # float
    predictions=predictions_cdf,      # (N,T), CDF
    time_grid=time_grid,
    ps=p_list,
)
print("slope:", slope)
print("p_list:", p_list)
print("obsevation_list:", obsevation_list)


slope: 1.4696149673286418
p_list: [0.1 0.3 0.5 0.7 0.9]
obsevation_list: [0.78232044 0.91254315 0.97099768 0.98948598 0.9941452 ]


# Test Well calibrated 

In [3]:
import numpy as np

rng = np.random.default_rng(42)
n_data = 1000
n_features = 10
X = rng.normal(size=(n_data, n_features))
beta = rng.normal(scale=0.3, size=n_features)    
shape_k = 1.3                                    
b0 = 0.2
lambda_i = np.exp(b0 + X @ beta)
U = rng.uniform(size=n_data)
event_times = lambda_i * (-np.log(U))**(1.0/shape_k)
censoring_times = rng.lognormal(mean=1.0, sigma=1.0, size=n_data)
event_indicators = event_times <= censoring_times  
observed_times   = np.minimum(event_times, censoring_times)

t_max = np.percentile(event_times, 99.5) * 1.5
n_times = 200
time_grid = np.linspace(0.0, t_max, n_times)
tt = time_grid[None, :]                 
ratio_k = (tt / lambda_i[:, None])**shape_k
predictions_cdf = 1.0 - np.exp(-ratio_k)            
predictions_surv = 1.0 - predictions_cdf            

p_list = (0.1,0.3,0.5,0.7,0.9)
p_list, obsevation_list, slope = calibration_slope_right_censor(
    event_indicators=event_indicators,     # bool
    observed_times=observed_times,    # float
    predictions=predictions_cdf,      # (N,T), CDF
    time_grid=time_grid,
    ps=p_list,
)
print("slope:", slope)
print("p_list:", p_list)
print("obsevation_list:", obsevation_list)


slope: 1.0721414563313214
p_list: [0.1 0.3 0.5 0.7 0.9]
obsevation_list: [0.10673575 0.34541336 0.55395683 0.75665399 0.9423329 ]


CoV

In [52]:
from IntervalCensor import cov_from_cdf_grid

cov_list = cov_from_cdf_grid(cdf = predictions_cdf, t_grid = time_grid)
print ('CoV:', np.nanmean(cov_list))

CoV: 0.7704234641985749


SurvAUPRC only for right and uncensor for now

In [47]:
from IntervalCensor import survival_auprc_right

scores = survival_auprc_right(event_indicators = event_indicators,
                              observed_times = observed_times,
                              predictions_cdf = predictions_cdf, 
                              time_grid = time_grid)
print (np.mean(scores))

0.5303564524194402
